In [1]:
!python -V

Python 3.5.6 :: Anaconda, Inc.


In [4]:
!pip install kfp --upgrade


In [8]:
import kfp
from kfp import gcp
from kfp import dsl

import os
from datetime import datetime
import kfp.compiler as compiler

client = kfp.Client(host='511b4fa793007e03-dot-us-central1.notebooks.googleusercontent.com')

In [9]:
# Set your output and project
YOUR_NAME = 'erwinh' #change to your name
EXPERIMENT_NAME = 'Scikit Learn V1.15 %s' % (YOUR_NAME) # change to your experiment name
PROJECT_NAME =  'erwinh-mldemo' # change this to your project name

In [10]:
experiment = client.create_experiment(EXPERIMENT_NAME)

In [17]:
#TODO change to KFP github
component_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/scikit-component/'

In [18]:
dummy_op = kfp.components.load_component_from_url(component_url_prefix + 'component.yaml') 

In [25]:
# Define a pipeline and create a task from a component:
@kfp.dsl.pipeline(name='KFP Scikit-learn', description='sklearn pipeline')
def my_pipeline(gcs_path_data = dsl.PipelineParam(name='pathdata', value='gs://erwinh-public-data/scikit/data/scikit-marketing-data.csv'),
               data_storage = dsl.PipelineParam(name='storage', value='GCS'),
               gcs_path_model = dsl.PipelineParam(name='pathoutput', value='gs://erwinh-public-data/kfp'), # change to your gcs path
               bigquery_table = dsl.PipelineParam(name='bqtable', value='kfp-primer-workshop.marketing_data.raw') # change to your gcs path
               ):
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        pathdata = gcs_path_data,
        storage = data_storage,
        pathoutput =gcs_path_model,
        bqtable = bigquery_table,
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))

In [26]:
pipeline_func = my_pipeline  
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [27]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run %s' %datetime.now()
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)